In [1]:
import time
import numpy as np
import tensorflow as tf

In [ ]:
# step 1: data preprocessing
# step 2: build model
# step 3: train model
# step 4: apply model - generate text

In [6]:
# load data

with open('/Users/pliu/Downloads/all/data_description.txt', 'r') as f:
    text = f.read()
type(text)

# get vocabulary
vocab = set(text)

# vocab-int mapping dict
vocab_to_int = {c:i for i, c in enumerate(vocab)}

# int-vocab mapping dict
int_to_vocab = dict(enumerate(vocab))

# encode text
encoded = np.array([vocab_to_int[c] for c in text], dtype=np.int32)

array([19, 37, 37, ..., 52, 64, 36], dtype=int32)

In [ ]:
def get_batches(arr, n_seqs, n_steps):
    """
    split array by batch and sequence (mini-batch)
    args:
        arr: the array to split
        n_seqs: num of sequence in a batch
        n_steps: length of each sequence
    """
    batch_size = n_seqs * n_steps
    n_batch = len(arr) // batch_size
    
    arr = arr[:n_batch*batch_size]
    
    arr = arr.reshape((n_seqs,-1))
    
    # generator
    for n in range(0, arr.shape[1], n_steps):
        x = arr[: n:n+n_steps]
        y = np.zeros_like(x)
        y[:, :-1], y[:, -1] = x[:, 1:], y[:, 0]
        yield x, y

In [ ]:
# build model

# layers: input, lstm, output
# loss
# optimizer

In [ ]:
# input layer
# size of input layer depends on the batch size
def build_inputs(n_seqs, n_steps):
    """
    build input layer
    args:
        n_seqs: num of sequence in each batch
        n_steps: length of char in each sequence
    """
    inputs = tf.placeholder(tf.int32, shape=(n_seqs, n_steps), name='inputs')
    targets = tf.placeholder(tf.int32, shape(n_seqs, n_steps), name='targets')
    
    # keep probability, for dropout
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    
    return inputs, targets, keep_prob

In [7]:
def build_lstm(lstm_size, num_layers, batch_size, keep_prob):
    """
    build lstm layer
    args:
        lstm_size: number of nodes in hidden layer
        num_layers: number of lstm layers
        batch_size: n_seq * n_steps
        keep_prob: parameter for dropout
    """
    # build a lstm cell
    cell = tf.nn.rnn_cell.BasicLSTMCell(lstm_size)
    
    # add dropout regularization
    drop = tf.nn.rnn_cell.DropoutWrapper(lstm, output_keep_prob=keep_prob)
    
    # add mutiple lstm cell
    cell = tf.nn.rnn_cell.MultiRNNCell([drop for _ in range(num_layers)])
    initial_state = cell.zero_state(batch_size, tf.float32)
    
    return cell, initial_state

In [8]:
def build_out(lstm_output, in_size, out_size):
    """
    build output layer
    args:
        lstm_output: output of lstm layer, a 3D array
        in_size: size of reshaped lstm
        out_size: size of softmax
    return:
        logits
        probability distribution after softmax
    """
    # reshape lstm output
    # concat output of lstm by column, 2D => 1D
    # [[1,2,3], [4,5,6]] => [1,2,3,4,5,6]
    seq_output = tf.concat(lstm_output, 1)
    # reshape
    x = tf.reshape(seq_output, [-1, in_size])
    
    # connect lstm to softmax layer
    with tf.variable_score('softmax'):
        softmax_w = tf.Variable(tf.truncated_normal([in_size, out_size],stddev=0.1))
        softmax_b = tf.Variable(tf.zeros(out_size))
        
    # calculate logits
    logits = tf.matmul(x, softmax_w) + softmax_b
    
    # get probability distribution from softmax layer
    out = tf.nn.softmax(logits, name='predictions')
    
    return out, logits   

In [ ]:
# loss
def build_loss(logits, targets, lsmt_size, num_classes):
    """
    calculate loss based on logits and targets
    args:
        logits: output of fully-connected layer, without softmax
        targets
        lsmt_size: number of nodes in lsmt layer
        num_classes: vocab_size, number of classes
    return:
    """
    # one-hot encoding to targets
    
    

In [ ]:
# step 3: train model
